In [1]:
spark

org.apache.spark.sql.SparkSession@555fbcc9

In [8]:
val myspark = spark

myspark = org.apache.spark.sql.SparkSession@30501b89


org.apache.spark.sql.SparkSession@30501b89

In [9]:
import myspark.implicits._

In [10]:
import org.apache.spark.sql.{functions => F}

import org.apache.spark.sql.{functions=>F}


In [11]:
val df = spark.read.json("ga_sample.json.gz")

df = [channelGrouping: string, customDimensions: array<struct<index:string,value:string>> ... 11 more fields]


[channelGrouping: string, customDimensions: array<struct<index:string,value:string>> ... 11 more fields]

In [15]:
df.printSchema

root
 |-- channelGrouping: string (nullable = true)
 |-- customDimensions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- index: string (nullable = true)
 |    |    |-- value: string (nullable = true)
 |-- date: string (nullable = true)
 |-- device: struct (nullable = true)
 |    |-- browser: string (nullable = true)
 |    |-- browserSize: string (nullable = true)
 |    |-- browserVersion: string (nullable = true)
 |    |-- deviceCategory: string (nullable = true)
 |    |-- flashVersion: string (nullable = true)
 |    |-- isMobile: boolean (nullable = true)
 |    |-- language: string (nullable = true)
 |    |-- mobileDeviceBranding: string (nullable = true)
 |    |-- mobileDeviceInfo: string (nullable = true)
 |    |-- mobileDeviceMarketingName: string (nullable = true)
 |    |-- mobileDeviceModel: string (nullable = true)
 |    |-- mobileInputSelector: string (nullable = true)
 |    |-- operatingSystem: string (nullable = true)
 |    |-- operati

In [16]:
df.cache.count

2556

In [27]:
df.groupBy($"device.browser").count.orderBy(F.desc("count")).show

+--------------------+-----+
|             browser|count|
+--------------------+-----+
|              Chrome| 1900|
|              Safari|  397|
|             Firefox|  101|
|   Internet Explorer|   54|
|                Edge|   23|
|          Opera Mini|   21|
|     Android Webview|   19|
|               Opera|   16|
|     Safari (in-app)|   10|
|          UC Browser|    6|
|           YaBrowser|    2|
|       Nokia Browser|    2|
|     Android Browser|    2|
|             Coc Coc|    2|
|Mozilla Compatibl...|    1|
+--------------------+-----+



In [12]:
val df2 = df.withColumn("revenue", $"totals.transactionRevenue"/1000000)

df2 = [channelGrouping: string, customDimensions: array<struct<index:string,value:string>> ... 12 more fields]


[channelGrouping: string, customDimensions: array<struct<index:string,value:string>> ... 12 more fields]

In [30]:
df2.filter("revenue is not null").select("revenue").show

+-------+
|revenue|
+-------+
|  35.29|
|  27.43|
|   1.99|
|  13.29|
|  11.96|
|   33.5|
|  200.0|
|  45.67|
|   2.99|
|  28.79|
|  43.64|
|   55.0|
|  48.77|
| 347.14|
| 982.73|
|   44.0|
|   22.4|
|  169.9|
|   26.0|
|  82.48|
+-------+
only showing top 20 rows



In [32]:
df2.
 groupBy("date").
 agg(
     F.count("*").alias("count"),
     F.sum("totals.visits"),
     F.sum("totals.pageviews"),
     F.sum("totals.transactions"),
     F.sum("revenue")).
show

+--------+-----+------------------+---------------------+------------------------+------------+
|    date|count|sum(totals.visits)|sum(totals.pageviews)|sum(totals.transactions)|sum(revenue)|
+--------+-----+------------------+---------------------+------------------------+------------+
|20170801| 2556|            2556.0|              10939.0|                    45.0|     8304.94|
+--------+-----+------------------+---------------------+------------------------+------------+



In [33]:
df2.
  groupBy("device.browser").agg(F.sum("totals.transactions").alias("total_transactions")).
  show


+--------------------+------------------+
|             browser|total_transactions|
+--------------------+------------------+
|Mozilla Compatibl...|              null|
|   Internet Explorer|              null|
|             Firefox|               1.0|
|           YaBrowser|              null|
|              Safari|               3.0|
|       Nokia Browser|              null|
|             Coc Coc|              null|
|     Android Browser|              null|
|          Opera Mini|              null|
|     Android Webview|              null|
|     Safari (in-app)|              null|
|              Chrome|              41.0|
|          UC Browser|              null|
|                Edge|              null|
|               Opera|              null|
+--------------------+------------------+



In [36]:
(df2
 .groupBy("trafficSource.source")
 .agg(F.count("*").alias("total_visits"), 
      F.sum("totals.bounces").alias("total_bounces"),
      (F.lit(100)*F.sum($"totals.bounces")/F.count($"*")).alias("bounce_rate")
     )
 
 ).show()

+--------------------+------------+-------------+------------------+
|              source|total_visits|total_bounces|       bounce_rate|
+--------------------+------------+-------------+------------------+
|           quora.com|           6|          4.0| 66.66666666666667|
|         youtube.com|         180|        139.0| 77.22222222222223|
|productforums.goo...|           1|          1.0|             100.0|
|                bing|           1|         null|              null|
|     lm.facebook.com|           2|         null|              null|
|         m.baidu.com|           1|          1.0|             100.0|
|      l.facebook.com|           3|          3.0|             100.0|
| ph.search.yahoo.com|           1|          1.0|             100.0|
|          google.com|          12|          9.0|              75.0|
|     docs.google.com|           1|         null|              null|
|        sashihara.jp|           1|          1.0|             100.0|
|analytics.google.com|          57

In [37]:
(df2
 .filter("totals.transactions >= 1")
 .groupBy("fullVisitorId")
 .agg( F.sum("totals.pageviews").alias("total_pagesviews_per_user"),
      (F.sum("totals.pageviews") / F.count("*")).alias("avg_pageviews_per_user"),
       F.count("*").alias("nb_visits")
     )
 .orderBy(F.desc("nb_visits"))
).show()

+-------------------+-------------------------+----------------------+---------+
|      fullVisitorId|total_pagesviews_per_user|avg_pageviews_per_user|nb_visits|
+-------------------+-------------------------+----------------------+---------+
|7420300501523012460|                    133.0|                  66.5|        2|
|3424918216070852195|                     43.0|                  43.0|        1|
| 454845221896711463|                     13.0|                  13.0|        1|
|1311003026247678016|                     14.0|                  14.0|        1|
|5371617068925994598|                     23.0|                  23.0|        1|
| 351681171177883933|                     45.0|                  45.0|        1|
|0377506370870345639|                     24.0|                  24.0|        1|
| 698297406423033664|                     16.0|                  16.0|        1|
|6403816382362616234|                     31.0|                  31.0|        1|
|1996186587769697436|       

In [38]:
(df2
 .filter("totals.transactions = 0")
 .groupBy("fullVisitorId")
 .agg( F.sum("totals.pageviews").alias("total_pagesviews_per_user"),
      (F.sum("totals.pageviews") / F.count("*")).alias("avg_pageviews_per_user"),
       F.count("*").alias("nb_visits")
     )
 .orderBy(F.desc("nb_visits"))
).show()

+-------------+-------------------------+----------------------+---------+
|fullVisitorId|total_pagesviews_per_user|avg_pageviews_per_user|nb_visits|
+-------------+-------------------------+----------------------+---------+
+-------------+-------------------------+----------------------+---------+



In [40]:
df2.select("hits").printSchema()

root
 |-- hits: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- appInfo: struct (nullable = true)
 |    |    |    |-- exitScreenName: string (nullable = true)
 |    |    |    |-- landingScreenName: string (nullable = true)
 |    |    |    |-- screenDepth: string (nullable = true)
 |    |    |    |-- screenName: string (nullable = true)
 |    |    |-- contentGroup: struct (nullable = true)
 |    |    |    |-- contentGroup1: string (nullable = true)
 |    |    |    |-- contentGroup2: string (nullable = true)
 |    |    |    |-- contentGroup3: string (nullable = true)
 |    |    |    |-- contentGroup4: string (nullable = true)
 |    |    |    |-- contentGroup5: string (nullable = true)
 |    |    |    |-- contentGroupUniqueViews1: string (nullable = true)
 |    |    |    |-- contentGroupUniqueViews2: string (nullable = true)
 |    |    |    |-- contentGroupUniqueViews3: string (nullable = true)
 |    |    |    |-- previousContentGroup1: string (nulla

In [48]:
(df2
 .select($"*",F.explode($"hits").alias("hit"))
 .select("fullVisitorId","visitId","visitNumber","hit.page.pagePath","hit.hitNumber")
 .orderBy("fullVisitorId","visitId")
).show(10,40)

+-------------------+----------+-----------+----------------------------------------+---------+
|      fullVisitorId|   visitId|visitNumber|                                pagePath|hitNumber|
+-------------------+----------+-----------+----------------------------------------+---------+
|0004915997121163857|1501620292|          1|  /google+redesign/shop+by+brand/youtube|        1|
|0013182148727187801|1501580378|          1|                                   /home|        1|
|0013182148727187801|1501580378|          1|                                   /home|        3|
|0013182148727187801|1501580378|          1|/google+redesign/bags/backpacks/water...|        5|
|0013182148727187801|1501580378|          1|                                   /home|        6|
|0013182148727187801|1501580378|          1|/google+redesign/drinkware/google+17o...|        7|
|0013182148727187801|1501580378|          1|                                   /home|        4|
|0013182148727187801|1501580378|        

In [52]:
(df2
 .select(F.explode($"hits").alias("hit"))
 .select(F.explode($"hit.product").alias("product"))
 .select(F.col("product"))
 
).printSchema()

root
 |-- product: struct (nullable = true)
 |    |-- customDimensions: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- customMetrics: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- isClick: boolean (nullable = true)
 |    |-- isImpression: boolean (nullable = true)
 |    |-- localProductPrice: string (nullable = true)
 |    |-- localProductRevenue: string (nullable = true)
 |    |-- productBrand: string (nullable = true)
 |    |-- productListName: string (nullable = true)
 |    |-- productListPosition: string (nullable = true)
 |    |-- productPrice: string (nullable = true)
 |    |-- productQuantity: string (nullable = true)
 |    |-- productRevenue: string (nullable = true)
 |    |-- productSKU: string (nullable = true)
 |    |-- productVariant: string (nullable = true)
 |    |-- v2ProductCategory: string (nullable = true)
 |    |-- v2ProductName: string (nullable = true)



In [54]:
(df2
 .select(F.explode($"hits").alias("hit"))
 .select(F.explode($"hit.product").alias("product"))
 .select("product.v2ProductCategory","product.v2ProductName")
 .distinct()
).show()

+--------------------+--------------------+
|   v2ProductCategory|       v2ProductName|
+--------------------+--------------------+
|   Home/Electronics/|   Google Flashlight|
|Home/Shop by Bran...|Android Wool Heat...|
|          Home/Bags/|Google Canvas Tot...|
| Home/Apparel/Kid's/|Google Infant Sho...|
|Home/Apparel/Wome...|Google Women's Co...|
|Home/Apparel/Kid'...|Google Infant Sho...|
| Home/Apparel/Men's/|Google Men's Colo...|
| Home/Shop by Brand/|YouTube Men's Vin...|
|Home/Accessories/...|Google Car Clip P...|
|Home/Apparel/Men'...|Google Men's 100%...|
|                 Fun|  Google Luggage Tag|
|Home/Apparel/Men'...|Google Men's  Zip...|
|Home/Apparel/Wome...|Google Women's We...|
| Home/Shop by Brand/|      Windup Android|
|Home/Apparel/Men'...|Google Men's Micr...|
|Home/Shop by Bran...|22 oz Android Bottle|
| Home/Apparel/Men's/|Google Men's 100%...|
|Home/Office/Noteb...| Google RFID Journal|
|Home/Office/Writi...|Ballpoint Stylus Pen|
|Home/Bags/Shoppin...|Collapsibl

In [56]:
(df2
 .select(F.col("device.browser").alias("browser"),F.explode($"hits").alias("hit"))
 .select($"browser",F.explode($"hit.product").alias("product"))
 .select("browser","product.v2ProductCategory","product.v2ProductName")

).show()

+-------+--------------------+--------------------+
|browser|   v2ProductCategory|       v2ProductName|
+-------+--------------------+--------------------+
| Chrome|   Home/Electronics/|Electronics Acces...|
| Chrome|   Home/Electronics/|   Google Flashlight|
| Chrome|   Home/Electronics/|Micro Wireless Ea...|
| Chrome|   Home/Electronics/|Google Car Clip P...|
| Chrome|   Home/Electronics/|Keyboard DOT Sticker|
| Chrome|   Home/Electronics/|Google Device Hol...|
| Chrome|   Home/Electronics/| Google Device Stand|
| Chrome|   Home/Electronics/|Google 2200mAh Mi...|
| Chrome|   Home/Electronics/|Google 4400mAh Po...|
| Chrome|   Home/Electronics/|Basecamp Explorer...|
| Chrome|   Home/Electronics/|   Rocket Flashlight|
| Chrome|   Home/Electronics/|Plastic Sliding F...|
| Chrome|Home/Apparel/Men'...|Google Tri-blend ...|
| Chrome|Home/Apparel/Men'...|Google Men's  Zip...|
| Chrome|Home/Apparel/Men'...|Google Men's Wate...|
| Chrome|Home/Apparel/Men'...|Google Men's Airf...|
| Chrome|Hom

In [60]:
(df2
 .select(F.col("device.browser").alias("browser"),F.explode($"hits").alias("hit"))
 .select($"browser",F.explode($"hit.product").alias("product"))
 .select("browser","product.v2ProductCategory","product.v2ProductName")
 .groupBy("v2ProductName")
 .pivot("browser")
 .count()
 .na.fill(0)
).show()

+--------------------+---------------+---------------+------+-------+----+-------+-----------------+------------------------+-------------+-----+----------+------+---------------+----------+---------+
|       v2ProductName|Android Browser|Android Webview|Chrome|Coc Coc|Edge|Firefox|Internet Explorer|Mozilla Compatible Agent|Nokia Browser|Opera|Opera Mini|Safari|Safari (in-app)|UC Browser|YaBrowser|
+--------------------+---------------+---------------+------+-------+----+-------+-----------------+------------------------+-------------+-----+----------+------+---------------+----------+---------+
|  Gift Card- $100.00|              0|              0|    17|      0|   0|      0|                0|                       0|            0|    0|         0|     0|              0|         0|        0|
|Google Men's Perf...|              0|              0|   313|      0|   1|     10|                1|                       0|            0|    1|         0|    25|              0|         1|      

In [61]:
df2.groupBy("device.browser").count().orderBy(F.desc("count")).show

+--------------------+-----+
|             browser|count|
+--------------------+-----+
|              Chrome| 1900|
|              Safari|  397|
|             Firefox|  101|
|   Internet Explorer|   54|
|                Edge|   23|
|          Opera Mini|   21|
|     Android Webview|   19|
|               Opera|   16|
|     Safari (in-app)|   10|
|          UC Browser|    6|
|     Android Browser|    2|
|           YaBrowser|    2|
|             Coc Coc|    2|
|       Nokia Browser|    2|
|Mozilla Compatibl...|    1|
+--------------------+-----+



In [66]:
val new_browser = F.when(F.col("device.browser").isin("Chrome","Safari","Firefox","Internet Explorer"),
                         F.col("device.browser"))
.otherwise("Other")

new_browser = CASE WHEN (device.browser IN (Chrome, Safari, Firefox, Internet Explorer)) THEN device.browser ELSE Other END


CASE WHEN (device.browser IN (Chrome, Safari, Firefox, Internet Explorer)) THEN device.browser ELSE Other END

In [68]:
(df2
 .select(new_browser.alias("browser"),F.explode($"hits").alias("hit"))
 .select($"browser",F.explode($"hit.product").alias("product"))
 .select("browser","product.v2ProductCategory","product.v2ProductName")
 .groupBy("v2ProductName")
 .pivot("browser")
 .count()
 .na.fill(0)
).show()

+--------------------+------+-------+-----------------+-----+------+
|       v2ProductName|Chrome|Firefox|Internet Explorer|Other|Safari|
+--------------------+------+-------+-----------------+-----+------+
|  Gift Card- $100.00|    17|      0|                0|    0|     0|
|Google Men's Perf...|   313|     10|                1|    4|    25|
|Android Women's S...|    46|      3|                2|    0|     7|
|Google Executive ...|    66|      4|                0|    0|     3|
|Google Women's Sc...|   128|      1|                1|    0|     8|
|Android Infant Sh...|    34|      1|                0|    0|     0|
| Red Shine 15 oz Mug|   173|     11|                1|    4|    13|
|Google Stylus Pen...|    60|      6|                0|    2|    10|
|YouTube Men's Sho...|   503|     44|               27|   63|   227|
|Google Women's Sh...|    14|      1|                0|    0|     3|
|Android RFID Journal|   263|     12|                4|    7|    27|
|Google Men's 100%...|    35|     

In [69]:
val df3 = (df2
 .select(new_browser.alias("browser"),F.explode($"hits").alias("hit"))
 .select($"browser",F.explode($"hit.product").alias("product"))
 .select("browser","product.v2ProductCategory","product.v2ProductName")
 .groupBy("v2ProductName")
 .pivot("browser")
 .count()
 .na.fill(0)
)

df3 = [v2ProductName: string, Chrome: bigint ... 4 more fields]


[v2ProductName: string, Chrome: bigint ... 4 more fields]

In [71]:
df3.filter("Firefox > Safari").show(truncate=false)

+--------------------------------------------------+------+-------+-----------------+-----+------+
|v2ProductName                                     |Chrome|Firefox|Internet Explorer|Other|Safari|
+--------------------------------------------------+------+-------+-----------------+-----+------+
|Google Executive Umbrella                         |66    |4      |0                |0    |3     |
|Android Infant Short Sleeve Tee Pink              |34    |1      |0                |0    |0     |
|Google Men's 100% Cotton Short Sleeve Hero Tee Red|35    |3      |0                |0    |2     |
|Waze Mobile Phone Vent Mount                      |72    |3      |0                |4    |1     |
|Google Men's Vintage Badge Tee White              |0     |1      |0                |0    |0     |
|Android Luggage Tag                               |225   |16     |1                |4    |14    |
|Waze Mood Original Window Decal                   |95    |4      |1                |1    |3     |
|Satin Bla

In [72]:
df3.filter("Firefox < Safari").show(truncate=false)

+-------------------------------------------------------+------+-------+-----------------+-----+------+
|v2ProductName                                          |Chrome|Firefox|Internet Explorer|Other|Safari|
+-------------------------------------------------------+------+-------+-----------------+-----+------+
|Google Men's Performance Full Zip Jacket Black         |313   |10     |1                |4    |25    |
|Android Women's Short Sleeve Badge Tee Dark Heather    |46    |3      |2                |0    |7     |
|Google Women's Scoop Neck Tee Black                    |128   |1      |1                |0    |8     |
|Red Shine 15 oz Mug                                    |173   |11     |1                |4    |13    |
|Google Stylus Pen w/ LED Light                         |60    |6      |0                |2    |10    |
|YouTube Men's Short Sleeve Hero Tee Black              |503   |44     |27               |63   |227   |
|Google Women's Short Sleeve Hero Tee Red Heather       |14    |

## RDD 

In [2]:
spark

org.apache.spark.sql.SparkSession@30501b89

In [3]:
val rdd = spark.parkContext.textFile("ga_sample.json.gz")

rdd = ga_sample.json.gz MapPartitionsRDD[1] at textFile at <console>:27


ga_sample.json.gz MapPartitionsRDD[1] at textFile at <console>:27

In [4]:
rdd.first

{"visitNumber":"1","visitId":"1501583974","visitStartTime":"1501583974","date":"20170801","totals":{"visits":"1","hits":"1","pageviews":"1","bounces":"1","newVisits":"1","sessionQualityDim":"1"},"trafficSource":{"campaign":"(not set)","source":"(direct)","medium":"(none)","adwordsClickInfo":{"criteriaParameters":"not available in demo dataset"}},"device":{"browser":"Chrome","browserVersion":"not available in demo dataset","browserSize":"not available in demo dataset","operatingSystem":"Android","operatingSystemVersion":"not available in demo dataset","isMobile":true,"mobileDeviceBranding":"not available in demo dataset","mobileDeviceModel":"not available in demo dataset","mobileInputSelector":"not available in demo dataset","mobileDeviceInfo":"not available in demo datase...

In [5]:
import play.api.libs.json._

In [6]:
val parsed = rdd.map(x => Json.parse(x))

parsed = MapPartitionsRDD[2] at map at <console>:30


MapPartitionsRDD[2] at map at <console>:30

In [7]:
parsed.first

{"visitNumber":"1","visitId":"1501583974","visitStartTime":"1501583974","date":"20170801","totals":{"visits":"1","hits":"1","pageviews":"1","bounces":"1","newVisits":"1","sessionQualityDim":"1"},"trafficSource":{"campaign":"(not set)","source":"(direct)","medium":"(none)","adwordsClickInfo":{"criteriaParameters":"not available in demo dataset"}},"device":{"browser":"Chrome","browserVersion":"not available in demo dataset","browserSize":"not available in demo dataset","operatingSystem":"Android","operatingSystemVersion":"not available in demo dataset","isMobile":true,"mobileDeviceBranding":"not available in demo dataset","mobileDeviceModel":"not available in demo dataset","mobileInputSelector":"not available in demo dataset","mobileDeviceInfo":"not ava...

### Query 1

In [13]:
df.groupBy("device.browser").count().orderBy(F.desc("count")).show(5)

+-----------------+-----+
|          browser|count|
+-----------------+-----+
|           Chrome| 1900|
|           Safari|  397|
|          Firefox|  101|
|Internet Explorer|   54|
|             Edge|   23|
+-----------------+-----+
only showing top 5 rows



In [18]:
parsed.map(x => (x \ "device" \ "browser").as[String]).first

Chrome

In [25]:
// RDD 
(parsed
 .map(x => ((x \ "device" \ "browser").as[String],1 ))
 .reduceByKey((x,y) => x + y)
 .top(5)(Ordering[(Int)].on(x=>x._2))
)

Array((Chrome,1900), (Safari,397), (Firefox,101), (Internet Explorer,54), (Edge,23))